In [ ]:
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE TypeFamilies #-}
{-# LANGUAGE AllowAmbiguousTypes #-}

import Control.Monad.ST
import Control.Monad.Primitive
import Data.Vector.Generic.Base
import qualified Data.Vector.Generic.Mutable as MV
import qualified Data.Vector.Generic as MG
import System.Random.MWC

In [ ]:
repeatSwap :: (PrimMonad m, Foldable (Mv (PrimState m)), MV.MVector v a) => v (PrimState m) a -> Gen (PrimState m) -> Int ->  m (v (PrimState m) a)
repeatSwap v g 0 = return v
repeatSwap v g n = do
    let n' = n - 1
    i <- uniformR (0, length v) g
    MV.swap v i n'
    repeatSwap v g n'

In [ ]:
-- body :: (PrimMonad m, Vector v a, Foldable (Mutable v (PrimState m))) => [a] -> Gen (PrimState m) -> m [a]
body :: (PrimMonad m, Foldable (v (PrimState m)), MV.MVector v a) => [a] -> Gen (PrimState m) -> m [a]
body xs g = do
    v <- MG.unsafeThaw $ MG.fromList xs
    repeatSwap v g $ length v
    MG.toList <$> MG.unsafeFreeze v

In [ ]:
randomST :: (Vector v a, Foldable (Mutable v s)) => [a] -> IO [a]
randomST xs = withSystemRandom . asGenST $ body xs

In [ ]:
:info randomST

In [ ]:
print <$> randomST [1, 2, 3, 4, 5, 6]

In [ ]:
shuffleList :: (PrimMonad m, Vector v a, Mutable v (PrimState m) a) => [a] -> IO [a]
shuffleList (a:[]) = return [a]
shuffleList xs =  withSystemRandom . asGenIO $ \g -> do
    v <- MG.unsafeThaw $ MG.fromList xs
    repeatSwap v g n
    r <- MG.toList <$> MG.unsafeFreeze v
    return r
        where
            n = length xs
            repeatSwap v g 0 = return v
            repeatSwap v g i = do
                let i' = i - 1
                j <- uniformR (0, n) g
                MV.swap v j i'
                repeatSwap v g i'